In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "0"

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platforms", 'cpu')


from ImageD11.parameters import AnalysisSchema
from ImageD11.columnfile import columnfile

In [2]:
pars = AnalysisSchema.from_default().geometry_pars_obj

In [3]:
pars.set('tilt_x', 0.00123)
pars.set('tilt_y', -0.0345)
pars.set('tilt_z', 0.02)
pars.set('chi', 1)
pars.set('wedge', -3)
pars.set('t_x', 1)
pars.set('t_y', 2)
pars.set('t_z', 3)

In [4]:
nrows = 100_000

fc = np.random.random(nrows) * 2048
sc = np.random.random(nrows) * 2048
om = np.random.random(nrows) * 360

In [5]:
cf = columnfile(new=True)
cf.nrows = nrows

cf.addcolumn(fc, 'fc')
cf.addcolumn(sc, 'sc')
cf.addcolumn(om, 'omega')

In [6]:
cf.parameters = pars

In [7]:
cf.updateGeometry()

In [8]:
cf.titles

['fc', 'sc', 'omega', 'xl', 'yl', 'zl', 'tth', 'eta', 'ds', 'gx', 'gy', 'gz']

In [9]:
gvecs = np.column_stack([cf.gx, cf.gy, cf.gz])

In [10]:
import ImageD11.transform, ImageD11.gv_general

In [11]:
import transform as mytrans

In [12]:
import importlib
importlib.reload(mytrans)

<module 'transform' from '/home/esrf/james1997a/Code/Anri/anri/sandbox/transform.py'>

# Full pipeline test: sc, fc, omega, origins (zeroes for now) to g-vectors

In [13]:
gvecs_me = mytrans.det_to_g(cf.sc, cf.fc, cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))), pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'), pars.get('wedge'), pars.get('chi'), pars.get('wavelength'))

In [14]:
gvecs_me

Array([[ 0.95948659,  0.10142665,  1.17956976],
       [ 0.58846085, -0.2468271 , -0.51533651],
       [ 0.04320291,  0.03133146,  0.44591103],
       ...,
       [ 0.28028584,  0.76705204, -0.25476143],
       [ 0.23840036, -0.50650501, -0.45499541],
       [-0.74953136,  0.91512474, -0.88148906]], dtype=float64)

In [15]:
gvecs

array([[ 0.95948659,  0.10142665,  1.17956976],
       [ 0.58846085, -0.2468271 , -0.51533651],
       [ 0.04320291,  0.03133146,  0.44591103],
       ...,
       [ 0.28028584,  0.76705204, -0.25476143],
       [ 0.23840036, -0.50650501, -0.45499541],
       [-0.74953136,  0.91512474, -0.88148906]])

In [16]:
assert np.allclose(gvecs_me, gvecs)

# Lab <-> Detector

In [17]:
%%time

xyz_id11 = ImageD11.transform.compute_xyz_lab((cf.sc, cf.fc), **pars.parameters)

CPU times: user 47.5 ms, sys: 0 ns, total: 47.5 ms
Wall time: 1.54 ms


In [18]:
%%time

xyz_me = mytrans.det_to_xyz_lab(cf.sc, cf.fc, pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

CPU times: user 4.97 ms, sys: 142 μs, total: 5.11 ms
Wall time: 113 μs


In [19]:
assert np.allclose(xyz_id11.T, xyz_me)

In [20]:
v_det_me = mytrans.xyz_lab_to_det(cf.xl, cf.yl, cf.zl, pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

In [21]:
assert np.allclose(v_det_me[:, 0], cf.sc)
assert np.allclose(v_det_me[:, 1], cf.fc)

# Lab <-> tth, eta, omega

In [22]:
tth_id11, eta_id11 = ImageD11.transform.compute_tth_eta_from_xyz(np.stack((cf.xl, cf.yl, cf.zl)), cf.omega, **pars.parameters)

In [23]:
tth_me, eta_me = mytrans.xyz_lab_to_tth_eta(jnp.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))), pars.get('wedge'), pars.get('chi'))

In [24]:
assert np.allclose(tth_me, tth_id11)
assert np.allclose(eta_me, eta_id11)

In [25]:
fc_id11, sc_id11 = ImageD11.transform.compute_xyz_from_tth_eta(cf.tth, cf.eta, cf.omega, **pars.parameters)

In [26]:
assert np.allclose(fc_id11, cf.fc)
assert np.allclose(sc_id11, cf.sc)

In [27]:
sc_me, fc_me = mytrans.tth_eta_omega_to_det(cf.tth, cf.eta, cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))),
                                          pars.get('wedge'), pars.get('chi'),  pars.get('wavelength'),
                                          pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                          pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                          pars.get('tilt_x'),
                                          pars.get('distance'),
                                          pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

In [28]:
assert np.allclose(sc_me, cf.sc)
assert np.allclose(fc_me, cf.fc)

In [29]:

dxyzl = mytrans.det_to_xyz_lab(sc, fc, pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

In [30]:
dxyzl.shape

(100000, 3)

In [31]:
t_id11 = ImageD11.transform.compute_grain_origins(cf.omega, pars.get('wedge'), pars.get('chi'), cf.xl, cf.yl, cf.zl)

In [32]:
t_me = mytrans.sample_to_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, pars.get('wedge'), pars.get('chi'))

In [33]:
assert np.allclose(t_id11, t_me.T)

In [34]:
%%time

tth_id11, (eta1_id11, eta2_id11), (omega1_id11, omega2_id11) = ImageD11.transform.uncompute_g_vectors(gvecs.T, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 470 ms, sys: 0 ns, total: 470 ms
Wall time: 31.6 ms


In [35]:
%%time

tth_me, (eta1_me, eta2_me), (omega1_me, omega2_me) = mytrans.g_to_tth_eta_omega(gvecs, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 9.33 s, sys: 208 ms, total: 9.54 s
Wall time: 264 ms


In [36]:
assert np.allclose(tth_id11, tth_me)
assert np.allclose(eta1_id11, eta1_me)
assert np.allclose(eta2_id11, eta2_me)
assert np.allclose(omega1_id11, omega1_me)
assert np.allclose(omega2_id11, omega2_me)

In [37]:
t_id11 = ImageD11.transform.compute_grain_origins(cf.omega, pars.get('wedge'), pars.get('chi'), cf.xl, cf.yl, cf.zl)

In [38]:
t_me = mytrans.sample_to_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, pars.get('wedge'), pars.get('chi'))

In [39]:
t_id11.shape

(3, 100000)

In [40]:
t_me.shape

(100000, 3)

In [41]:
assert np.allclose(t_id11, t_me.T)

In [42]:
%%time

tth_id11, (eta1_id11, eta2_id11), (omega1_id11, omega2_id11) = ImageD11.transform.uncompute_g_vectors(gvecs.T, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 445 ms, sys: 21.9 ms, total: 467 ms
Wall time: 31.4 ms


In [43]:
tth_id11

array([25.04572209, 13.40417926,  7.32731607, ..., 13.98287433,
       11.78277976, 24.23321229])

In [44]:
%%time

tth_me, (eta1_me, eta2_me), (omega1_me, omega2_me) = mytrans.g_to_tth_eta_omega(gvecs, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 9.28 ms, sys: 312 μs, total: 9.59 ms
Wall time: 208 μs


In [45]:
tth_me

Array([25.04572209, 13.40417926,  7.32731607, ..., 13.98287433,
       11.78277976, 24.23321229], dtype=float64)

In [46]:
assert np.allclose(tth_id11, tth_me)
assert np.allclose(eta1_id11, eta1_me)
assert np.allclose(eta2_id11, eta2_me)
assert np.allclose(omega1_id11, omega1_me)
assert np.allclose(omega2_id11, omega2_me)

In [47]:
eta1_me

Array([ 37.32982109, 129.85362524,   2.47897353, ..., 108.10008206,
       130.01702547, 127.92389259], dtype=float64)

In [48]:
eta1_id11

array([ 37.32982109, 129.85362524,   2.47897353, ..., 108.10008206,
       130.01702547, 127.92389259])

In [49]:
# test k vector computation
k_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))

In [50]:
k_id11

array([[-0.33042703, -0.09572685, -0.02869676, ..., -0.10412891,
        -0.07404563, -0.30965151],
       [ 0.86022399,  0.64323832,  0.00372509, ..., -0.80709301,
        -0.54956059,  1.16805578],
       [ 1.21372129, -0.49986233,  0.44816003, ..., -0.26379994,
        -0.46141444, -0.84618861]])

In [51]:
k_me = mytrans.tth_eta_to_k(cf.tth, cf.eta, pars.get('wavelength'))

In [52]:
k_me

Array([[-0.33042703,  0.86022399,  1.21372129],
       [-0.09572685,  0.64323832, -0.49986233],
       [-0.02869676,  0.00372509,  0.44816003],
       ...,
       [-0.10412891, -0.80709301, -0.26379994],
       [-0.07404563, -0.54956059, -0.46141444],
       [-0.30965151,  1.16805578, -0.84618861]], dtype=float64)

In [53]:
k_id11.shape

(3, 100000)

In [54]:
k_me.shape

(100000, 3)

In [55]:
assert np.allclose(k_id11.T, k_me)

In [56]:
# test computation of angles for g-vectors

W =  mytrans.wedgemat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))
pre = (C @ W).T

oms_id11 = ImageD11.gv_general.g_to_k(gvecs.T, pars.get('wavelength'), axis=[0,0,1], pre=pre, post=None)

In [57]:
oms_id11

(array([-104.31007188, -122.03708361,  -94.5061092 , ...,  -26.43846002,
        -164.63160328,   25.80441422]),
 array([ 113.3714804 ,   76.055541  ,  133.49612977, ...,  168.21771307,
          31.06076335, -124.86253601]),
 array([ True,  True,  True, ...,  True,  True,  True]))

In [58]:
oms_me = mytrans.omega_solns_for_g(gvecs, pars.get('wavelength'), np.array([0,0,1]), pre, jnp.eye(3))

In [59]:
oms_me

(Array([-104.31007188, -122.03708361,  -94.5061092 , ...,  -26.43846002,
        -164.63160328,   25.80441422], dtype=float64),
 Array([ 113.3714804 ,   76.055541  ,  133.49612977, ...,  168.21771307,
          31.06076335, -124.86253601], dtype=float64),
 Array([ True,  True,  True, ...,  True,  True,  True], dtype=bool))

In [60]:
assert np.allclose(oms_id11[0], oms_me[0])
assert np.allclose(oms_id11[1], oms_me[1])
assert np.allclose(oms_id11[2], oms_me[2])

In [61]:
# now use k vectors to test g_from_k

In [62]:
# test k vector computation
k_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))
g_id11 = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [63]:
g_id11

array([[ 0.95948659,  0.58846085,  0.04320291, ...,  0.28028584,
         0.23840036, -0.74953136],
       [ 0.10142665, -0.2468271 ,  0.03133146, ...,  0.76705204,
        -0.50650501,  0.91512474],
       [ 1.17956976, -0.51533651,  0.44591103, ..., -0.25476143,
        -0.45499541, -0.88148906]])

In [64]:
g_me = mytrans.lab_to_sample(k_id11.T, cf.omega, pars.get('wedge'), pars.get('chi'))

In [65]:
g_me

Array([[ 0.95948659,  0.10142665,  1.17956976],
       [ 0.58846085, -0.2468271 , -0.51533651],
       [ 0.04320291,  0.03133146,  0.44591103],
       ...,
       [ 0.28028584,  0.76705204, -0.25476143],
       [ 0.23840036, -0.50650501, -0.45499541],
       [-0.74953136,  0.91512474, -0.88148906]], dtype=float64)

In [66]:
assert np.allclose(g_id11.T, g_me)

In [67]:
g_id11_trans = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [68]:
g_id11_trans

array([[ 0.95948659,  0.58846085,  0.04320291, ...,  0.28028584,
         0.23840036, -0.74953136],
       [ 0.10142665, -0.2468271 ,  0.03133146, ...,  0.76705204,
        -0.50650501,  0.91512474],
       [ 1.17956976, -0.51533651,  0.44591103, ..., -0.25476143,
        -0.45499541, -0.88148906]])

In [69]:
# C @ W -Z works

W =  mytrans.wedgemat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))

post = C @ W

g_id11_gvgeneral = ImageD11.gv_general.k_to_g(k_id11, cf.omega, axis=np.array([0., 0., -1]), pre=None, post=post)

In [70]:
g_id11_gvgeneral

array([[ 0.95948659,  0.58846085,  0.04320291, ...,  0.28028584,
         0.23840036, -0.74953136],
       [ 0.10142665, -0.2468271 ,  0.03133146, ...,  0.76705204,
        -0.50650501,  0.91512474],
       [ 1.17956976, -0.51533651,  0.44591103, ..., -0.25476143,
        -0.45499541, -0.88148906]])

In [71]:
assert np.allclose(g_id11_trans, g_id11_gvgeneral)

In [72]:
g_id11 = ImageD11.transform.compute_g_vectors(cf.tth, cf.eta, cf.omega, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

In [73]:
g_id11

array([[ 0.95948659,  0.58846085,  0.04320291, ...,  0.28028584,
         0.23840036, -0.74953136],
       [ 0.10142665, -0.2468271 ,  0.03133146, ...,  0.76705204,
        -0.50650501,  0.91512474],
       [ 1.17956976, -0.51533651,  0.44591103, ..., -0.25476143,
        -0.45499541, -0.88148906]])

In [74]:
assert np.allclose(gvecs, g_id11.T)

In [75]:
g_me = mytrans.tth_eta_omega_to_g(cf.tth, cf.eta, cf.omega, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

In [76]:
g_me

Array([[ 0.95948659,  0.10142665,  1.17956976],
       [ 0.58846085, -0.2468271 , -0.51533651],
       [ 0.04320291,  0.03133146,  0.44591103],
       ...,
       [ 0.28028584,  0.76705204, -0.25476143],
       [ 0.23840036, -0.50650501, -0.45499541],
       [-0.74953136,  0.91512474, -0.88148906]], dtype=float64)

In [77]:
assert np.allclose(gvecs, g_me)

In [78]:
assert np.allclose(gvecs, mytrans.sample_to_lab(mytrans.lab_to_sample(gvecs, cf.omega, pars.get('wedge'), pars.get('chi')), cf.omega, pars.get('wedge'), pars.get('chi')))

In [79]:
# g-vectors to (tth, eta, omega)
tth, [eta_one, eta_two], [omega1, omega2] = mytrans.g_to_tth_eta_omega(gvecs, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

In [80]:
assert np.allclose(tth, cf.tth)
assert all(np.logical_or(np.isclose(cf.eta, eta_one), np.isclose(cf.eta, eta_two)))
assert all(np.logical_or(np.isclose(cf.omega, omega1 % 360), np.isclose(cf.omega, omega2 % 360)))

In [81]:
# (tth, eta, omega) to g-vectors
gvecs_loop = mytrans.tth_eta_omega_to_g(tth, eta_one, omega1, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))
assert np.allclose(gvecs_loop, gvecs)

In [82]:
# (tth, eta, omega) to g-vectors
gvecs_loop = mytrans.tth_eta_omega_to_g(tth, eta_two, omega2, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))
assert np.allclose(gvecs_loop, gvecs)

In [83]:
k_me = mytrans.xyz_lab_to_k(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))), pars.get('wedge'), pars.get('chi'), pars.get('wavelength'))

In [84]:
k_me

Array([[-0.33042703,  0.86022399,  1.21372129],
       [-0.09572685,  0.64323832, -0.49986233],
       [-0.02869676,  0.00372509,  0.44816003],
       ...,
       [-0.10412891, -0.80709301, -0.26379994],
       [-0.07404563, -0.54956059, -0.46141444],
       [-0.30965151,  1.16805578, -0.84618861]], dtype=float64)

In [85]:
sc_me, fc_me = mytrans.k_to_det(k_me, cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))), pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

In [86]:
sc_me

Array([ 354.00009457, 1414.97430753,  855.55399372, ..., 1274.96106592,
       1391.70152482, 1655.88918657], dtype=float64)

In [87]:
xyz_me = mytrans.k_to_xyz_lab(k_me, cf.omega, jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z'))), pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

In [88]:
assert np.allclose(xyz_me, np.column_stack((cf.xl, cf.yl, cf.zl)))

In [89]:
xyz_me

Array([[149952.71657758,  40518.2435823 ,  57171.38496362],
       [152934.08967949,  28778.24817801, -22360.66257928],
       [152057.90860147,    161.88613095,  19555.43606685],
       ...,
       [153875.0366588 , -36422.73851274, -11901.13067906],
       [153940.73824932, -24591.29444115, -20645.06465661],
       [153018.03486437,  55777.840309  , -40403.94382808]],      dtype=float64)